# Set Up

In [23]:
import pandas as pd
import altair as alt
import NBAStatsFetcher
import importlib
from NBAStatsFetcher import StatsFetcher

In [24]:
pd.set_option('display.max_columns', None)

# Load Data

In [25]:
importlib.reload(NBAStatsFetcher)

fetch_player_data_totals = StatsFetcher(
    api_name='player_data_totals',
    api_url='http://rest.nbaapi.com/api/playerdatatotals/query',
    season=2025,
    sort_by="PlayerName",
    ascending=False,
    delay=0.5,
    page_size=50
)

fetch_player_data_advanced = StatsFetcher(
    api_name='player_data_advanced',
    api_url='http://rest.nbaapi.com/api/playerdataadvanced/query',
    season=2025,
    sort_by="PlayerName",
    ascending=False,
    delay=0.5,
    page_size=50
)

player_data_totals_df = fetch_player_data_totals.get_dataframe()
player_data_advanced_df = fetch_player_data_advanced.get_dataframe()

2025-04-20 12:53:37,136 - Fetching NBA player stats from player_data_totals...
Teams processed: 100%|██████████| 30/30 [00:20<00:00,  1.47it/s]
2025-04-20 12:53:57,497 - No failed requests.
2025-04-20 12:53:57,508 - Fetching NBA player stats from player_data_advanced...
Teams processed: 100%|██████████| 30/30 [00:19<00:00,  1.52it/s]
2025-04-20 12:54:17,247 - No failed requests.


In [26]:
# Columns in advanced that are not in totals (excluding the key 'playerId')
unique_advanced_cols = [
    col for col in player_data_advanced_df.columns
    if col not in ['playerId', 'team'] and col not in player_data_totals_df.columns
]

# Keep only playerId + the unique columns from advanced
advanced_subset = player_data_advanced_df[['playerId', 'team'] + unique_advanced_cols]

# Now merge cleanly — no duplicates or suffixes!
stats_df = pd.merge(player_data_totals_df, advanced_subset, on=['playerId', 'team'], how='inner')


In [27]:
# Only keep data from the last team the player played on in the season
stats_df = stats_df.sort_values(['playerId', 'id'], ascending=[True, False])
stats_df = stats_df.drop_duplicates(subset='playerId', keep='first')

In [28]:
assert stats_df[stats_df['playerName'].str.contains('Luka Dončić', case=False, na=False)]['team'].iloc[0] == 'LAL', "Luka Dončić is not in team 'LAL'"
assert stats_df[stats_df['playerName'].str.contains('Anthony Davis', case=False, na=False)]['team'].iloc[0] == 'DAL', "Anthony Davis is not in team 'DAL'"

# Data Profiling

In [29]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 570 entries, 416 to 273
Data columns (total 53 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  570 non-null    int64  
 1   playerName          570 non-null    object 
 2   position            570 non-null    object 
 3   age                 570 non-null    int64  
 4   games               570 non-null    int64  
 5   gamesStarted        570 non-null    int64  
 6   minutesPg           570 non-null    float64
 7   fieldGoals          570 non-null    int64  
 8   fieldAttempts       570 non-null    int64  
 9   fieldPercent        566 non-null    float64
 10  threeFg             570 non-null    int64  
 11  threeAttempts       570 non-null    int64  
 12  threePercent        539 non-null    float64
 13  twoFg               570 non-null    int64  
 14  twoAttempts         570 non-null    int64  
 15  twoPercent          560 non-null    float64
 16  effectFgPer

In [30]:
missing_values = stats_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values

fieldPercent        4
threePercent       31
twoPercent         10
effectFgPercent     4
ftPercent          31
tsPercent           4
threePAR            4
ftr                 4
turnoverPercent     3
dtype: int64

# Data Cleaning

In [31]:
# Impute missing values
stats_df['fieldPercent'] = stats_df['fieldPercent'].fillna(
    (stats_df['fieldGoals'] / stats_df['fieldAttempts']).fillna(0)
)

stats_df['threePercent'] = stats_df['threePercent'].fillna(
    (stats_df['threeFg'] / stats_df['threeAttempts']).fillna(0)
)

stats_df['twoPercent'] = stats_df['twoPercent'].fillna(
    (stats_df['twoFg'] / stats_df['twoAttempts']).fillna(0)
)

stats_df['effectFgPercent'] = stats_df['effectFgPercent'].fillna(
    ((stats_df['fieldGoals'] + (stats_df['threeFg'] / 2)) / stats_df['fieldAttempts']).fillna(0)
)

stats_df['ftPercent'] = stats_df['ftPercent'].fillna(
    (stats_df['ft'] / stats_df['ftAttempts']).fillna(0)
)

stats_df['tsPercent'] = stats_df['tsPercent'].fillna(
    (stats_df['points'] / (2 * (stats_df['fieldAttempts'] + 0.44 * stats_df['ftAttempts']))).fillna(0)
)

stats_df['threePAR'] = stats_df['threePAR'].fillna(
    (stats_df['threeAttempts'] / stats_df['fieldAttempts']).fillna(0)
)

stats_df['ftr'] = stats_df['ftr'].fillna(
    (stats_df['ftAttempts'] / stats_df['fieldAttempts']).fillna(0)
)

stats_df['turnoverPercent'] = stats_df['turnoverPercent'].fillna(
    (stats_df['ftAttempts'] / stats_df['fieldAttempts']).fillna(0)
)

stats_df['turnoverPercent'] = stats_df['turnoverPercent'].fillna(
    ((stats_df['turnovers'] / (stats_df['fieldAttempts'] + (0.44 * stats_df['ftAttempts']) + stats_df['turnovers'])) * 100).fillna(0)
)

In [32]:
missing_values = stats_df.isnull().sum()
missing_values = missing_values[missing_values > 0]

if missing_values.empty:
    print("✅ No missing values")
else:
    print("⚠️ Missing values found:")
    display(missing_values)

✅ No missing values


In [33]:
# Rename minutesPg to totalMinutes
stats_df = stats_df.rename(columns={'minutesPg': 'totalMinutes'})

In [34]:
# Create new columns
stats_df['mpg'] = round(stats_df['totalMinutes'] / stats_df['games'], 1).fillna(0)
stats_df['ppg'] = round(stats_df['points'] / stats_df['games'], 1).fillna(0)
stats_df['rpg'] = round(stats_df['totalRb'] / stats_df['games'], 1).fillna(0)
stats_df['apg'] = round(stats_df['assists'] / stats_df['games'], 1).fillna(0)
stats_df['bpg'] = round(stats_df['blocks'] / stats_df['games'], 1).fillna(0)
stats_df['spg'] = round(stats_df['steals'] / stats_df['games'], 1).fillna(0)

In [35]:
stats_df.to_csv('player_stats_2025.csv', index=False)